In [ ]:
# Code to get the data set for the OBIS workshop

In [2]:
# step 1 URL for Santa Cruz
urlapi='https://ifcb.caloos.org/api/'
url='https://ifcb.caloos.org/santa-cruz-municipal-wharf/'

In [10]:
# should create code to get data based upon feed of URL and site
import xarray as xr
import numpy as np
import pandas as pd
import ifcb_api_access

In [30]:
# Lets get some bin_id's
bin_ids=ifcb_api_access.get_bins_in_range("2023-07-17","2023-07-18")
print(bin_ids)

0     uz-municipal-wharf/D20230717T000942_IFCB104
1     uz-municipal-wharf/D20230717T003329_IFCB104
2     uz-municipal-wharf/D20230717T005715_IFCB104
3     uz-municipal-wharf/D20230717T012101_IFCB104
4     uz-municipal-wharf/D20230717T014447_IFCB104
5     uz-municipal-wharf/D20230717T020833_IFCB104
6     uz-municipal-wharf/D20230717T023218_IFCB104
7     uz-municipal-wharf/D20230717T025605_IFCB104
8     uz-municipal-wharf/D20230717T031951_IFCB104
9     uz-municipal-wharf/D20230717T034337_IFCB104
10    uz-municipal-wharf/D20230717T040723_IFCB104
11    uz-municipal-wharf/D20230717T043109_IFCB104
12    uz-municipal-wharf/D20230717T045455_IFCB104
13    uz-municipal-wharf/D20230717T051841_IFCB104
14    uz-municipal-wharf/D20230717T054226_IFCB104
15    uz-municipal-wharf/D20230717T060613_IFCB104
16    uz-municipal-wharf/D20230717T062959_IFCB104
17    uz-municipal-wharf/D20230717T065345_IFCB104
18    uz-municipal-wharf/D20230717T071731_IFCB104
19    uz-municipal-wharf/D20230717T074116_IFCB104


In [31]:
# get the meta data for a bin ID 
# Note need to remove all the values prior to "D"
bin_id=bin_ids[0][19:]
metadatavals=ifcb_api_access.get_ifcb_metadata(bin_ids[0][19:])

In [32]:
len(bin_ids)

58

In [33]:
# Do we have autoclass data for this file
#ib=np.arange(0,len(bin_ids))
ib=np.arange(0,2)
for i in ib:
    bin_id=bin_ids[i][19:]
    test=ifcb_api_access.bin_has_autoclass(bin_id)
    print(test)

True
True


In [39]:
# Not all "bins" have autoclass scores
class_names_file='c:/users/flbahr/class_names_matched_to_WoRMS_cencoos_classifier_20240412.csv'
class_names=pd.read_csv(class_names_file)

In [40]:
target_labels=class_names['Class Name']

In [41]:
# get the api autoclass file for the data
bin_url=url+bin_id+'_class_scores.csv' # defined above
autoclass=pd.read_csv(bin_url)

In [45]:
autoclass

,pid,Akashiwo,Alexandrium_singlet,Amy_Gony_Protoc,Asterionellopsis,Boreadinium,Centric,Ceratium,Chaetoceros,Ciliates,...,Scrip_Het,Skeletonema,Thalassionema,Thalassiosira,Tiarina,Tintinnid,Tontonia,Torodinium,Tropidoneis,Vicicitus
0,D20230717T003329_IFCB104_00002,2.400000e-07,0.000000e+00,0.000000e+00,6.000000e-08,6.000000e-08,7.000000e-07,6.260000e-06,0.000003,0.000000,...,6.000000e-08,6.000000e-08,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e-07,0.000000e+00,0.000000e+00,6.000000e-08,0.000000e+00
1,D20230717T003329_IFCB104_00003,6.400000e-06,3.953000e-04,5.300000e-06,8.000000e-07,5.000000e-07,3.754000e-02,1.300000e-06,0.000116,0.010056,...,1.842500e-03,2.400000e-07,0.000000e+00,5.440000e-03,0.000000e+00,0.000000e+00,1.335000e-05,1.694000e-04,0.000000e+00,5.400000e-07
2,D20230717T003329_IFCB104_00004,1.135000e-03,1.166000e-03,1.147000e-02,4.300000e-06,2.300000e-06,4.343000e-01,5.390000e-03,0.022220,0.005154,...,4.856000e-03,1.530000e-05,0.000000e+00,7.940000e-03,0.000000e+00,1.071000e-04,5.290000e-04,1.384000e-02,0.000000e+00,3.000000e-07
3,D20230717T003329_IFCB104_00005,2.500000e-05,8.000000e-07,4.080000e-04,4.554000e-05,2.400000e-07,1.411000e-04,2.330000e-03,0.004425,0.000020,...,1.010000e-05,3.200000e-06,1.000000e-07,2.660000e-05,0.000000e+00,1.418000e-03,1.910000e-05,2.900000e-06,6.000000e-08,0.000000e+00
4,D20230717T003329_IFCB104_00006,2.150000e-05,5.188000e-03,1.596000e-02,4.600000e-06,6.425000e-05,1.873000e-01,5.120000e-03,0.001621,0.099730,...,6.380000e-02,4.300000e-06,0.000000e+00,3.244000e-03,7.000000e-07,4.000000e-07,6.588000e-03,3.695000e-05,6.000000e-08,4.650000e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3005,D20230717T003329_IFCB104_03007,1.000000e-07,9.000000e-07,0.000000e+00,0.000000e+00,3.600000e-07,2.540000e-05,5.400000e-07,0.000002,0.000000,...,0.000000e+00,0.000000e+00,0.000000e+00,9.000000e-07,0.000000e+00,0.000000e+00,6.000000e-08,0.000000e+00,6.000000e-08,1.130000e-06
3006,D20230717T003329_IFCB104_03008,0.000000e+00,0.000000e+00,2.000000e-07,9.000000e-07,0.000000e+00,2.000000e-07,2.056000e-05,0.000002,0.000000,...,5.000000e-07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
3007,D20230717T003329_IFCB104_03009,0.000000e+00,3.760000e-06,5.300000e-06,1.681000e-04,2.700000e-06,8.800000e-06,1.350000e-03,0.004505,0.000002,...,6.060000e-05,1.710000e-05,0.000000e+00,0.000000e+00,0.000000e+00,1.020000e-05,1.250000e-06,6.600000e-07,6.000000e-08,0.000000e+00
3008,D20230717T003329_IFCB104_03010,2.000000e-07,0.000000e+00,1.430000e-06,4.000000e-07,0.000000e+00,6.600000e-07,4.165000e-04,0.000001,0.000000,...,1.000000e-07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.000000e-08,6.000000e-08,0.000000e+00,0.000000e+00,0.000000e+00


In [46]:
details=ifcb_api_access.get_bin_details(bin_id)

In [49]:
# Now have metadata values and details
# metadatavals and details
# autoclass for the bin
# bin_id="D20230717T003329_IFCB104"

In [50]:
# get the features file
feature_filename=bin_id+'_features.csv'
features=pd.read_csv(url+feature_filename)

In [51]:
features

,roi_number,Area,B180,B90,Bflip,Biovolume,BoundingBox_xwidth,BoundingBox_ywidth,ConvexArea,ConvexPerimeter,...,HOG79,HOG80,HOG81,Area_over_PerimeterSquared,Area_over_Perimeter,H90_over_Hflip,H90_over_H180,Hflip_over_H180,summedConvexPerimeter_over_Perimeter,rotated_BoundingBox_solidity
0,2,54574.0,0.368497,0.181079,0.962857,3.758912e+06,575,224,90723.669102,1302.338204,...,0.308797,0.361135,0.316723,0.022128,34.750909,35.205691,2.031675,0.057709,0.829286,0.427089
1,3,1267.0,0.918046,0.825847,0.930654,3.857876e+04,48,36,1327.957732,129.915465,...,0.164193,0.264625,0.485808,0.067791,9.267747,2.203961,2.194031,0.995494,0.950295,0.699945
2,4,1912.0,0.880233,0.866455,0.888182,6.354290e+04,55,52,2247.137266,170.274532,...,0.169124,0.354672,0.445574,0.053102,10.076294,1.283148,1.064763,0.829806,0.897352,0.647341
3,5,7878.0,0.810080,0.622953,0.956329,2.892267e+05,167,95,11551.292600,433.585199,...,0.426714,0.309362,0.332592,0.027104,14.612376,14.979906,2.152259,0.143676,0.804228,0.572041
4,6,3090.0,0.867661,0.787869,0.878365,1.457461e+05,77,63,3718.942534,222.885069,...,0.266490,0.260658,0.400797,0.040571,11.196557,1.960536,1.802546,0.919415,0.807620,0.665013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3005,3007,22180.0,0.631605,0.244364,0.885482,1.064719e+06,425,148,44007.229629,948.459258,...,0.361883,0.362313,0.316244,0.013340,17.201133,4.970794,1.958042,0.393909,0.735553,0.328495
3006,3008,6073.0,0.690331,0.587547,0.705320,1.755594e+05,115,127,10820.651364,394.302728,...,0.146769,0.224298,0.602977,0.021600,11.453202,1.088068,1.021528,0.938845,0.743624,0.417509
3007,3009,6198.0,0.816049,0.554603,0.910856,1.916049e+05,155,115,9919.963414,405.926828,...,0.367091,0.250991,0.289471,0.020879,11.375633,11.090338,3.437931,0.309993,0.745027,0.535430
3008,3010,7818.0,0.784216,0.598491,0.936812,2.823157e+05,179,86,11944.866111,439.732222,...,0.294159,0.545535,0.219234,0.024856,13.940040,11.349296,2.251563,0.198388,0.784073,0.502023


In [52]:
# okay now we have most of the pieces 
# start to construct the tables?
# don't know if the random bin id I picked has the species we are looking for above the threshold

In [54]:
# need to load the class threshold file also
threshold=pd.read_csv('c:/users/flbahr/class_thresholds_cencoos_classifier_20240412.csv')

In [56]:
# files that are unique to the deployments are the class name file and the threshold file
# these can NOT be extracted from the api as far as we know

In [57]:
threshold.head(3)

,Class Name,Threshold
0,Akashiwo,0.95
1,Alexandrium_singlet,0.49
2,Amy_Gony_Protoc,0.35


In [58]:
class_names.head(3)

,Class Name,Class ID,Common Name,AphiaID,URN,Grouping,HAB,Description,Manual Classifier Notes,Example Images
0,Akashiwo,0,Akashiwo sanguinea,232546.0,urn:lsid:marinespecies.org:taxname:232546,Dinoflagellate,Y,"Monophyletic, marine dinoflagellate","large single cells, kidney-shaped when viewed ...",NaN
1,Alexandrium_singlet,1,Alexandium sp.,109470.0,urn:lsid:marinespecies.org:taxname:109470,Dinoflagellate,Y,A. catenella and others,"Originally class was broken out for singles, d...",http://akashiwo.oceandatacenter.ucsc.edu:8000/...
2,Amy_Gony_Protoc,2,"Amylax, Gonyaulax or Protoceratium",109428.0,urn:lsid:marinespecies.org:taxname:109428,Dinoflagellate,Y,Descision was made not to distinguish between ...,“Feet” and points visible.,NaN


In [63]:
# okay we have the class name but not the intended_worms_taxon
# do I need intended_worms_taxon?  We have the AphialID for the class from another file
a2=autoclass
a2=a2.set_index(['pid'],append=False)
a2.index.name=None

In [65]:
a3=a2.stack(future_stack=True)

In [68]:
ik=np.arange(0,len(a3))
for r in ik:
    junk=str(a3[r:r+1]).split()
    if r==0:
        pid=[junk[0]]
        myclass=[junk[1]]
        score=[float(junk[2])]
    else:
        pid.append(junk[0])
        myclass.append(junk[1])
        score.append(float(junk[2]))

In [69]:
bigtable=pd.DataFrame({'pid':pid,'class':myclass,'score':score})

In [72]:
[l1,l2]=threshold.shape

In [76]:
ix=np.arange(0,l1)
zz=0
for kk in ix:
    id1=bigtable['class']==threshold['Class Name'][kk]
    subtable=bigtable[['pid','class','score']][id1]
    id2=subtable['score'] >= threshold['Threshold'][kk]
    smalltable=subtable[['pid','class','score']][id2]
    if smalltable.size > 0:
        if zz==0:
            classtable=smalltable
            zz=zz+1
        else:
            classtable=pd.concat([classtable, smalltable],ignore_index=True)
            #classtable.append(smalltable)

In [79]:
classtable.head(3)

,pid,class,score
0,D20230717T003329_IFCB104_00033,Akashiwo,0.999
1,D20230717T003329_IFCB104_00107,Akashiwo,1.000
2,D20230717T003329_IFCB104_00311,Akashiwo,1.000


In [80]:
summarytable=classtable.groupby('class').count()

In [85]:
#summarytable.rename(columns={'score':'occurrences'})
summarytable=summarytable.rename(columns={'score':'occurrences'}).drop('pid',axis=1)

In [86]:
# missing occurrence to add 0's